## Logistic Regression

Notebook with implementation of the Logistic Regression algorithm to predict victory in Dota 2

-------------------------------------------------------------------------------------------------------------------------------

Useful functions to use to explore the data and preprocessing steps before feeding the data into the algorithm:

* df.columns : to see the names of the columns (i.e., features)
* df.dtype : to see the types in the data
* data.head()
* data.info()
* df.describe()

## Score blowout matches

In [2]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn import metrics
import statistics as st
from sklearn import preprocessing

In [3]:
# Directory for the score blowout group
cwd = os.getcwd()
root_directory = os.path.dirname(cwd)
score_blowout_data_dir = root_directory + "\\model_features_pre-match\\score_blowout\\"

### Exploration and preprocessing of the data

In [3]:
# Read data
feature_score_blowout_df = pd.read_csv(score_blowout_data_dir + "dota2_score_blowout_features.csv")

In [ ]:
# Drop first column (match id)
feature_score_blowout_df = feature_score_blowout_df.drop(['match_id'], axis=1)

# Print feature names
feature_score_blowout_df.columns

In [ ]:
feature_score_blowout_df.head()

In [ ]:
feature_score_blowout_df.info()

In [ ]:
#Fill in missing data with the median value of the feature
feature_score_blowout_df = feature_score_blowout_df.fillna(feature_score_blowout_df.median())

In [ ]:
feature_score_blowout_df['rad_first_pick'] = feature_score_blowout_df['rad_first_pick'].astype(int)

### Model building, training and evaluation

In [8]:
# Import logistic regression library
from sklearn.linear_model import LogisticRegression

In [9]:
# Split into features (X) and label (y)
X, y = feature_score_blowout_df.iloc[:,:-1],feature_score_blowout_df.iloc[:,-1]

In [10]:
features = [c for c in feature_score_blowout_df.columns if c != 'win_label']
target = 'win_label'

In [11]:
# Define the number of folds for the k-fold cross-validation
kfolds = KFold(n_splits=10, shuffle=True)

In [12]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# Scale features
columns_to_scale = list(range(238,456))

In [13]:
auc = list()

mm_scaler = preprocessing.MinMaxScaler()

for train_idx, test_idx in kfolds.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]  
    
    X_train.iloc[:,columns_to_scale] = mm_scaler.fit_transform(X_train.iloc[:,columns_to_scale])
    logreg.fit(X_train, y_train)
    
    X_test.iloc[:,columns_to_scale] = mm_scaler.transform(X_test.iloc[:,columns_to_scale])
    y_pred_proba =logreg.predict_proba(X_test)[::,1]
    auc.append(metrics.roc_auc_score(y_test, y_pred_proba))

'Median AUC: {:.04f}'.format(st.median(auc))

C:\Users\markos-ece\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\markos-ece\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

C:\Users\markos-ece\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\markos-ece\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

'Median AUC: 0.7749'